In [ ]:
import cvxpy as cp
import numpy as np

# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    Box_vertices = np.array(Box.vertices_list())
    return Box_vertices

# We write a function to compute minimum distance between two polyhedra, and return the points
def min_dist_between_polyhedra(P1, P2):
    # At first, we extract the linear constraints from both polyhedra written as Ax + b >= 0
    Ab1 = list(P1.Hrepresentation())
    Ab2 = list(P2.Hrepresentation())
    b1, A1 = ([a[0] for a in Ab1], [a[1:] for a in Ab1])
    b2, A2 = ([a[0] for a in Ab2], [a[1:] for a in Ab2])
    b1, A1, b2, A2 = np.array(b1), np.array(A1), np.array(b2), np.array(A2)
    # print('b1 = ', b1)
    # print('A1 = ', A1)
    # print('b2 = ', b2)
    # print('A2 = ', A2)

    # The dimension of the polyhedra are the number of columns in Matrix A
    n = len(A1[0])
    print('n = ', n)

    # We define two vectors of same dimensions
    X1 = cp.Variable(n)
    X2 = cp.Variable(n)

    # Next, we write the constraints for the two polyhedra P1 and P2 such that vector X1 in P1 and X2 in P2
    constraints = [A1@X1 >= -b1, A2@X2 >=-b2]

    # Finally, we write the objective function which is to minimize the infinity norm of any two points in P1 and P2
    objective = cp.Minimize(cp.norm(X1-X2))

    # Problem formulation
    prob = cp.Problem(objective, constraints)

    # Solve the problem
    prob.solve()

    # Finally, we have the minimum distance and the two vectors,
    val, x1, x2 = prob.value, X1.value, X2.value
    # print('Minimum distance = ', val)
    # print('Vector in P1 = ', x1)
    # print('Vector in P2', x2)
    return val, x1, x2

def min_dist_pol_vertices(P1,P2):
    dist_list = [[np.linalg.norm(x1-x2), x1, x2] for x1 in P1 for x2 in P2]
    minval = min(x[0] for x in dist_list)
    result = [x for x in dist_list if x[0]==minval]
    val, x1, x2 = result[0]
    return val, x1, x2

# Next, we write a function to compute the maximum distance of a vector and a polyhedron
def max_dist_vector_to_polyhedron2(v, P):
    dist_list = [np.linalg.norm(v-x) for x in P]
    d = max(x for x in dist_list)
    print('dist_list = ', dist_list)
    return d, [0,0]


# Next, we write a function to compute the maximum distance of a vector and a polyhedron
def max_dist_vector_to_polyhedron(v, P):
    X = P.vertices_list()
    dist_list = [(np.linalg.norm(v-x), x) for x in X]
    d, vx = max(dist_list)
    return d, vx

# Minkowski sum function
def minkowski_pol(V1, V2):
    V3 = []
    for v1 in V1:
        for v2 in V2:
            n = v1.size
            v = np.zeros(n)
            for i in range(n):
                v[i] = v1[i] + v2[i]
            V3.append(v)
    # print(V3)
    V4 = np.array(V3)
    return V4

def convex_hull(V1, V2):
    V3 = V1 + V2
    return V3

def linear_trans(M, V):
    # V1 = P1.vertices_list()
    # V2 = P2.vertices_list()
    V1 = [np.dot(M,v) for v in V]
    return V1

In [ ]:
import time
start_time = time.time()

# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 1: ")
exp_num = v
# Evaluate the inputs
if (int(v)==1):
    input_file = open(r"inputs_new/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs_new/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs_new/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs_new/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()

# Taking the input values
xn, xu = n, u
P0 = [np.hstack((x,u,1)) for x in X0 for u in U0]
# W = W.vertices_list()

# Making matrix M for twice X
I = np.identity(xn)
M = Matrix(np.vstack((I,I)))

# Converting K into K = [[I 0] [0 K]]
K1 = [[0 for i in range(xn)]for j in range(xn)]
KI = np.identity(xn)
K1 = np.hstack((KI, K1))
K2 = [[0 for i in range(xn)] for j in range(xu)]
K2 = np.hstack((K2,K))
Kp = np.array(np.vstack((K1,K2)))

# Take the center of the box
vbox2 = [0]*(2*n)
vbox = [0]*n


# Opening a file to write the outputs
exp_string = "outputs/alg3_new/ex" + exp_num + ".csv"
f = open(exp_string, "a")
f.write('\nNew Experiment:\n')

f.write('Iteration, Num_Cons(Xout), Num_Ver(Xout), Volume(Xout), Runtime\n')
# Running algorithm 3
P = P0
l, u = T0[0], T0[1]
iter = 6
for i in range(iter):
        print('i = ',i)
        AlC = A*l + C
        BlD = B*l + D
        AuC = A*u + C
        BuD = B*u + D
        # print('AlC = ', AlC, 'BlD = ', BlD, 'E = ', E)
        J1 = np.hstack((AlC,BlD,E))
        J2 = np.hstack((AuC,BuD,E))
        # print('J1 = ', J1)
        # print('J2 = ', J2)
        # print('P = ', P)
        X1 = linear_trans(J1,P)
        # print('X1 = ', X1)
        X2 = linear_trans(J2,P)
        # print('X2 = ', X2)
        X = convex_hull(X1,X2)
        # print('X = ', X)
        ep = c/l        
        print('ep = ', ep)
        X_twice = linear_trans(M,X)
        # print('X_twice = ', X_twice)
        rbox2 = [0]*n +[ep]*n
        Box_b = make_box(vbox2, rbox2)
        Xh = minkowski_pol(X_twice, Box_b)
        # print("Xh = ", Xh)
        P1 = linear_trans(Kp,Xh)
        # print('P1 = ', P1)
        P = [np.hstack((p,1)) for p in P1]
        # print('P = ', P)
        rbox = [ep]*n
        Box_a = make_box(vbox,rbox)
        Xpp_ver = minkowski_pol(X, Box_a)        
        Xpp = Polyhedron(vertices=Xpp_ver, backend='ppl', base_ring=QQ)
        # print('Xpp = ', Xpp_ver)
        print('W = ', W.vertices_list())
        l, x1, w1 = min_dist_pol_vertices(Xpp_ver, W.vertices_list())
        u, x2 = max_dist_vector_to_polyhedron2(w1, Xpp_ver)        
        l = 100 if (l > 100) else l
        u = 100 if (u > 100) else u
        print('l, x1, w1, u, x2')
        print(l,x1,w1,u,x2)
        X_poly = Polyhedron(vertices=X, backend='ppl', base_ring=QQ)
        curr_time = (time.time()-start_time)
        msg = str((i+1, len(list(X_poly.Hrepresentation())), len(list(X_poly.vertices())), "{:e}".format(float(X_poly.volume())), curr_time))
        msg = msg[1:-1]
        print(msg)
        f.write(msg+'\n')
        
# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3
f.write('\nTotal runtime = '+str(round(total_time_in_ms, 4))+' ms for '+str(iter)+' iterations\n')

# Close the file
f.close()


print('X = ', X)
